# Импорт библиотек

In [ ]:
!pip install -q pymorphy2 # установка лемматизатора pymorphy2

     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 8.2 MB 11.6 MB/s 


In [ ]:
!pip install tensorflow==2.3 # устанавливаем версию tf 2.3 и делаем restart runtime

     |████████████████████████████████| 320.4 MB 48 kB/s 
     |████████████████████████████████| 2.9 MB 33.9 MB/s 
     |████████████████████████████████| 20.1 MB 12.1 MB/s 
     |████████████████████████████████| 459 kB 42.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:


In [ ]:
import numpy as np # Библиотека для работы с массивами данных
from tensorflow.keras.models import Model, load_model # Импортируем Model, load_model - метод, что загружает предобученную сеть
import re # Имортируем чтобы работать с строками
from tensorflow.keras.preprocessing.text import Tokenizer # Метод, который поволяет работать с текстами и конвертирует их в последовательности (индексов)
# Импорт слоёв нейронных сетей
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Conv1DTranspose, Lambda
from tensorflow.keras import backend as K # Импортируем, чтобы высчитать dice_coef(ошибку)
from tensorflow.keras.optimizers import Adam, Adadelta # Импортируем оптимизаторы
from tensorflow.keras import utils # Импортируем для работы с категориальными данными
import tensorflow
from google.colab import files # Импорт для работы с файлами
import matplotlib.pyplot as plt # Импорт для отрисовывания графиков
from gensim.models import word2vec # Импортируем gensim
import os # Импортируем для работы с системными файлами
import pandas as pd # Импортируем для работы с Массивами данных(таблицами - Датафреймами)
import time # Имортируем, чтобы высчитать время работы каких-либо процессов
import nltk #Natural language toolkit - Инструментарий естественного языка
from nltk.stem import WordNetLemmatizer  # Импортируем для работы с леммами
import pymorphy2 # Импортируем для работы с леммами

nltk.download('wordnet') # Скачиваем сетку слов для лемматизации

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Функции для работы с текстами


Превращаем договоры в листы слов

## **Лист тегов и их значений**

**`<s1>`** - Условия

**`<s2>`** - Запреты

**`<s3>`** - Цена

**`<s4>`** - Условия (относительно дат)

**`<s5>`** - Штрафы

**`<s6>`** - Адреса и местоположения


In [ ]:
# Считываем текст из файла и удаляем пунктуационные знаки препинания и еще дополнительные ненужные нам знаки
def readText(fileName):
  f = open(fileName, 'r') # Открываем наш файл и считываем наши данные
  text = f.read() # Помещаем наш текст в переменную
  # Определяем, какие знаки будут удалены
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";",'•','%']

  for dS in delSymbols: # Каждый знак из нашего списка будет удалён из списка
    text = text.replace(dS, " ") # Удаляем знак, посредством замены этого знака на пробел

  # Выискиваем дополнительные знаки в текстах посредством паттернов и определяем на что их заменять
  text = re.sub("[.]", " ", text)
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  # split  метод разделит текст на последовательность слов при помощи пробелов
  # (а их может быть в одном месте очень много после наших удалений знаков), которые будет являться разделителями.
  # Используя join метод, мы обратно соберём последовательности слов в текст
  text = ' '.join(text.split()) 

  text = text.lower() # Конвертируем текст к нижнему регистру
  return text # Возвращаем текст

# Конвертируем исходный текст в лист слов с начальной формой 
def text2Words(text):
  morph = pymorphy2.MorphAnalyzer() # Инициализируем инструмент для работы с морфемами и более
  words = text.split(' ') #  Разделяем текст по средством пробелов
  docs = [morph.parse(word)[0].normal_form for word in words] # Превращаем каждое слово в элемент списка
  return docs # Возвращаем полученный документ

# Read the data




In [ ]:
!unzip -q '/content/Договора432-20220108T074358Z-001.zip' -d '/content/contracts' 

error:  cannot create /content/contracts/Договора432/Договор страхования ответственности производителя или исполнителя за причинение вреда потребителю вследствие недостатков товара, работы, услуги.txt
        File name too long
error:  cannot create /content/contracts/Договора432/Договор страхования имущества (страхователь - физическое или юридическое лицо, в пользу выгодоприобретателя - физического или юридического лица).txt
        File name too long
error:  cannot create /content/contracts/Договора432/Договор страхования имущества (страхователь - физическое или юридическое лицо_ в пользу выгодоприобретателя - физического или юридического лица_ сострахование).txt
        File name too long
error:  cannot create /content/contracts/Договора432/Договор сберегательного страхования жизни (страхователь - физическое лицо, в пользу застрахованного - физического лица, страховой случай - дожитие застрахованного до момента вступления в брак).txt
        File name too long


In [ ]:
directory = '/content/contracts/Договора432/' # Путь к папке с договорами 
os.listdir(directory)[250:255] # Выводим на экран название 5 договоров

['Договор перестрахования (эксцедента суммы).txt',
 '3 (2).txt',
 '5.ДОГОВОР АРЕНДЫ ПРИЦЕПА ДЛЯ ЛЕГКОВОГО АВТОМОБИЛЯ.txt',
 '15.ДОГОВОР ИМУЩЕСТВЕННОГО НАЙМА (АРЕНДЫ) ОБОРУДОВАНИЯ.txt',
 'ДОГОВОР ФИНАНСИРОВАНИЯ ПОД УСТУПКУ ДЕНЕЖНОГО ТРЕБОВАНИЯ (ФАКТОРИНГ).txt']

In [ ]:
print('Всего', len(os.listdir(directory)), 'договора')

Всего 428 договора


In [ ]:
curTime = time.time() # Запоминаем текущее время
agreements = [] # Лист, что будет хранить все наши договоры
for filename in os.listdir(directory): # Проходимся по каждому договору
  txt = readText(directory + filename) # Считываем текст в договоре
  if txt != '': # Если договор не пустой
    agreements.append(readText(directory + filename)) # Превращаем файл в текст и добавляем его в agreements
print('Загрузка файла заняла: ', round(time.time() - curTime, 2), 'с.')

Загрузка файла заняла:  0.34 с.


In [ ]:
n = 3
print(os.listdir(directory)[n]) # Выводим названия n-ого договора
agreements[n]                   # Текст n-ого договора

dogovor-podryada-na-remontnostroitelnie-raboti.txt


'договор подряда на ремонтно строительные работы г г в лице действующего на основании именуемый в дальнейшем подрядчик с одной стороны и в лице действующего на основании именуемый в дальнейшем заказчик с другой стороны именуемые в дальнейшем стороны заключили настоящий договор в дальнейшем договор о нижеследующем 1 предмет договора 1 1 <s1> заказчик поручает а подрядчик принимает на себя обязательство в установленный договором срок выполнить ремонтно строительные работы в помещениях заказчика по адресу согласно техническому заданию проектной документации смете приведенной в приложении 1 а заказчик обязуется создать подрядчику необходимые условия для выполнения работ принять выполненные работы и уплатить обусловленную договором цену </s1> 1 2 виды работ объем работ единичные расценки на работы составляющие предмет настоящего договора определяются сметой приложение 1 к настоящему договору и являются неотъемлемой частью настоящего договора 1 3 все работы в рамках настоящего договора произ

In [ ]:
docs_full = [] # Лист будет содержать все договоры как списки документов
curTime = time.time() # Запоминаем текущее время
for i in range(len(agreements)): # Проходимся по каждому договору
  docs_full.append(text2Words(agreements[i])) # Превращаем договор в лист слов и добавляем его в docs_full
print('Превращение заняло: ', round(time.time() - curTime, 2), 's.')

Превращение заняло:  203.78 s.


In [ ]:
print(len(docs_full)) # Выводим число записей в наборе данных 

# Выбираем итоговое количество данных для обучающей/проверочной и тестовой выборках
docs = docs_full[0:-10]
docsToTest = docs_full[-10:]

418


In [ ]:
print('Число текстов в для тестовой проверки в конце ноутбука:', len(docsToTest)) 
print('Число договоров для обучающей и проверочной выборках:',len(docs)) 
print('Число слов в первом договоре:', len(docs[10]))

Число текстов в для тестовой проверки в конце ноутбука: 10
Число договоров для обучающей и проверочной выборках: 408
Число слов в первом договоре: 5895


In [ ]:
print('Пример текста обычного:')
print(agreements[4][:62], '\n')
print('Тот же текст, но представленный ввиде лемм:')
print(docs[4][:10])

Пример текста обычного:
договор на участие в уставных фондах г г в лице действующего н 

Тот же текст, но представленный ввиде лемм:
['договор', 'на', 'участие', 'в', 'уставный', 'фонд', 'г', 'г', 'в', 'лицо']


# Превращение текстов в последовтельность индексов: создание xTrain


In [ ]:
# lower=True - приводит к нижнему регистру все слова
# char_level=False - говорит токенайзеру не инициализировать отдельно каждую букву как токен  
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)

tokenizer.fit_on_texts(docs_full) # Скармливаем тексты токенайзеру
clean_voc = {}                    # Создаем пустой словарь

for item in tokenizer.word_index.items(): # Превращаем лист в словарь
  clean_voc[item[0]] = item[1]            # Мы меняем местами элеметны кортежа

In [ ]:
print('Словарь все слов по их частотноти:') 
print(clean_voc, '\n')
print('Длина словаря:', len(clean_voc))

tag = '<s1>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь все слов по их частотноти:
{'в': 1, 'договор': 2, 'и': 3, '2': 4, '1': 5, 'по': 6, 'сторона': 7, 'с': 8, 'настоящий': 9, 'на': 10, '3': 11, '<s1>': 12, '</s1>': 13, 'не': 14, '4': 15, 'или': 16, 'за': 17, 'он': 18, 'случай': 19, 'о': 20, '5': 21, '<s3>': 22, '</s3>': 23, 'работа': 24, 'право': 25, 'срок': 26, '<s4>': 27, '</s4>': 28, 'от': 29, 'при': 30, 'они': 31, '6': 32, 'другой': 33, 'быть': 34, 'имущество': 35, 'арендатор': 36, 'действовать': 37, 'день': 38, 'лицо': 39, 'условие': 40, 'для': 41, '7': 42, 'к': 43, 'если': 44, 'дальнейший': 45, 'п': 46, 'работник': 47, 'который': 48, 'обязательство': 49, 'а': 50, 'законодательство': 51, 'порядок': 52, 'арендодатель': 53, 'сумма': 54, 'размер': 55, 'действие': 56, 'свой': 57, 'счёт': 58, 'соответствие': 59, 'указанный': 60, 'страховой': 61, 'именовать': 62, 'из': 63, 'мочь': 64, 'основание': 65, 'исполнение': 66, 'один': 67, 'обязанность': 68, 'обязать': 69, 'иной': 70, 'год': 71, 'трудовой': 72, 'г': 73, 'до': 74, 'заказчик'

In [ ]:
# Превращает текст в последовательность индексов согласно словарю частотности
tok_agreem = tokenizer.texts_to_sequences(docs) # Обучающий выборка в последовтельность индексов

print("Взгляните на фрагмент обучающей выборки:")
print("Исходный текст:              ", docs[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreem[4][:20], '\n')

Взгляните на фрагмент обучающей выборки:
Исходный текст:               ['договор', 'на', 'участие', 'в', 'уставный', 'фонд', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'банк', 'с', 'один', 'сторона']
Тот же текст, но как последовательность индексов:  [2, 10, 556, 1, 809, 781, 73, 73, 1, 39, 37, 10, 65, 62, 1, 45, 104, 8, 67, 7] 



# Создание yTrain



Каждый индекс (слово) из договора будет соответсвовать вектору, чьи элементы соотвествуют категориям тегов

In [ ]:
#  Собираем лист индексов и их мультилейбл класссификации
def getXYSamples(tok_agreem, tags_index):
  tags01 = [] # лист тегов
  indices = [] # лист индексов
 
  for agreement in tok_agreem: # Проходимся по каждому договору-листу
    tag_place = [0, 0, 0, 0, 0, 0] # создаём вектор [0,0,0,0,0,0] 
    for ex in agreement: # Проходимся по каждому слову из договора
        if ex in tags_index: # Проверяем, если индекс слова соотвествует индексу одного из тегов
          place = np.argwhere(tags_index==ex) # Запоминаем позицию тега в этом листе тегов
          if len(place)!=0: # Проверяем, если тек в самом деле есть
            if place[0][0]<6: # Первые 6 тегов - открывающие теги
              tag_place[place[0][0]] = 1    # устанавливаем значение 1
            else: 
              tag_place[place[0][0] - 6] = 0  # Все остальные теги закрывающие, так что мы меняем в соответсвующем месте в векторе на 0
        else:          
          tags01.append(tag_place.copy()) # Добавляем в наш лист тегов новые вектора мульти-лейбл классификации. 
                                          # В конце концов у нас будет большой лист всех вектор-тегов
          indices.append(ex) # Добавляем индекс-слова в лист индексов

  return indices, tags01

# Получаем лист слов из листа индексов
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) #  Берёт ключ и значение из словаря и меняет их местами
  docs = [reverse_word_map.get(letter) for letter in x] # Положим их в лист
  return docs # Возвращает текст

In [ ]:
#<s1> - Условия договора
#<s2> - Запреты
#<s3> - Стоимость
#<s4> - Условия (относительно дат)
#<s5> - Штрафы
#<s6> - Адреса и местоположения

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] # Лист открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] # Лист закрывающих тегов
tags_index.extend(closetags) # конкатенируем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем инексы всех тегов из словря частотности
print('Индексы всех тегов:', tags_index)

Индексы всех тегов: [ 12 352  22  27 117 276  13 358  23  28 118 268]


In [ ]:
curTime = time.time()
xData, yData = getXYSamples(tok_agreem,tags_index) # Получаем теги и создаём листы с ними
decoded_text = reverseIndex(clean_voc, xData) # Чтобы создать лист для эмбеддинга, нам нужно превратить лист индексов обратно в лист слов
print('Превращение заняло: ', round(time.time() - curTime, 2), 'с.')

Превращение заняло:  3.87 с.


In [ ]:
print('длина xData:', len(xData))
print('длина yData:', len(yData))

print('Посмотрим на исходные слова: ', docs[0][50:80])
print('Сдекодированые слова:', decoded_text[50:80])
print('Часть xData:     ', xData[50:80])
print('Часть yData:     ', yData[50:80])

длина xData: 399215
длина yData: 399215
Посмотрим на исходные слова:  ['нижеследующий', '1', 'я', 'гр', 'менять', 'принадлежащий', 'я', 'двигатель', 'автомашина', 'марка', 'год', 'выпуск', 'на', 'двигатель', 'автомашина', 'марка', 'год', 'выпуск', 'принадлежащий', 'гр', '2', 'указанный', 'двигатель', 'принадлежать', 'гр', 'на', 'основание', 'технический', 'паспорт', 'выдать']
Сдекодированые слова: ['нижеследующий', '1', 'я', 'гр', 'менять', 'принадлежащий', 'я', 'двигатель', 'автомашина', 'марка', 'год', 'выпуск', 'на', 'двигатель', 'автомашина', 'марка', 'год', 'выпуск', 'принадлежащий', 'гр', '2', 'указанный', 'двигатель', 'принадлежать', 'гр', 'на', 'основание', 'технический', 'паспорт', 'выдать']
Часть xData:      [205, 5, 1138, 518, 1709, 417, 1138, 1286, 1552, 1653, 71, 941, 10, 1286, 1552, 1653, 71, 941, 417, 518, 4, 60, 1286, 501, 518, 10, 65, 199, 249, 165]
Часть yData:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0

# Разделение выборки на окна


In [ ]:
# Создание выборки из индексов
def getSetFromIndices(wordIndices, xLen, step): 
  xBatch = [] # Лист, хранящий фрагменты текста
  wordsLen = len(wordIndices) # получаем длину текста
  index = 0 #  стартовый индекс
  
  while (index + xLen <= wordsLen): # Пока сумма индекса и окна меньше или равно чем число слов в выборке
    xBatch.append(wordIndices[index:index+xLen]) # Добавляем X в лист, что содержит наши фрагменты(окна) текста
    index += step # Смещаем наше индекс на наш шаг(step)

  return xBatch #  Лист фрагментов текста

In [ ]:
xLen = 256 # Длина каждого окна
step = 30 # шаг 
embeddingSize = 300 # Число измерений вектор-пространства для каждого нашего слова 

In [ ]:
#  Генерируем выборки с параметрами наших окон
xTrain = getSetFromIndices(decoded_text, xLen, step) # Последовательность индексов с окном xLen слов
yTrain = getSetFromIndices(yData, xLen, step) # Последовательность индексов с окном xLen от тегов

In [ ]:
print('Длина xTrain:', len(xTrain))
print('Длина yTrain:', len(yTrain))
print('Длина примера из xTrain:',len(xTrain[0]))
print('Длина примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')

print('Первый пример xTrain:', xTrain[0][step-5:step+5])
print('Второй пример xTrain:', xTrain[1][:10])

Длина xTrain: 13299
Длина yTrain: 13299
Длина примера из xTrain: 256
Длина примера из yTrain: 256 

Пример xTrain ['договор', 'о', 'обмен', 'двигатель', 'г', 'г', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'сторона', '1', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'сторона', '2', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', '1', 'я', 'гр', 'менять', 'принадлежащий', 'я', 'двигатель', 'автомашина', 'марка', 'год', 'выпуск', 'на', 'двигатель', 'автомашина', 'марка', 'год', 'выпуск', 'принадлежащий', 'гр', '2', 'указанный', 'двигатель', 'принадлежать', 'гр', 'на', 'основание', 'технический', 'паспорт', 'выдать', 'гаи', 'от', 'год', 'и', 'гр', 'на', 'основание', 'технический', 'паспорт', 'выдать', 'гаи', 'от', 'год

In [ ]:
# Превращаем текст в последовательность индексов согласно словарю частотности
tok_agreemTest = tokenizer.texts_to_sequences(docsToTest) # Обучающий текст в индексы

print("Посмотрим на фрагмент тестового текста:")
print("Исходный текст:              ", docsToTest[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreemTest[4][:20], '\n')

xDataTest, yDataTest = getXYSamples(tok_agreemTest,tags_index) # Распознаём теги и создаём лист, что их хранят
decoded_text = reverseIndex(clean_voc, xDataTest) # Чтобы создать лист для эмбеддинга, нам нужно превратить лист индексов обратно в лист слов
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Сдекодированные текст:', decoded_text[50:80])
print('Часть xDataTest:     ', xDataTest[50:80])
print('Часть yDataTest:     ', yDataTest[50:80])

xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Число измерений вектор-пространства для каждого нашего слова 

# Генерируем выборки с определёнными параметрами
xTest = getSetFromIndices(decoded_text, xLen, step) # Последовательность индексов с окном Xlen слов
yTest = getSetFromIndices(yDataTest, xLen, step) # # Последовательность индексов с окном Xlen от тегов

print('Длина xTest:', len(xTest))
print('Длина yTest:', len(yTest))
print('Длина примера xTest:',len(xTest[0]))
print('Длина примера yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Посмотрим на фрагмент тестового текста:
Исходный текст:               ['договор', 'о', 'залог', 'имущество', 'с', 'оставление', 'заложить', 'имущество', 'у', 'залогодатель', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший']
Тот же текст, но как последовательность индексов:  [2, 20, 136, 35, 8, 2652, 289, 35, 193, 189, 73, 73, 1, 39, 37, 10, 65, 62, 1, 45] 

Длина xDataTest: 13611
Длина yDataTest: 13611
Сдекодированные текст: ['1', 'ссудодатель', 'передавать', 'в', 'безвозмездный', 'временной', 'пользование', 'ссудополучатель', 'принадлежащий', 'ссудодатель', 'на', 'право', 'собственность', 'автомобиль', 'марка', 'выпуск', 'год', 'двигатель', 'кузов', 'цвет', 'номерной', 'знак', 'паспорт', 'транспортный', 'средство', 'выдать', 'год', 'далее', 'автомобиль', '1']
Часть xDataTest:      [5, 387, 298, 1, 580, 527, 167, 379, 417, 387, 10, 25, 130, 316, 1653, 941, 71, 1286, 1451, 1982, 1819, 1289, 249, 345, 84, 165, 71, 246, 316, 5]
Часть yDataTest:      

# Функции создания моделей


## Функция, что считает ошибку


In [ ]:
# Функция нужна для точности
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

In [ ]:
# Функция, что создаёт линейную свёрточную сеть
def create_Conv1d(xLen, embeddingSize): 
  text_input_layer = Input((xLen,embeddingSize)) 
  text_layer = Conv1D(32, 3, padding='same',activation='relu')(text_input_layer)
  text_layer = Conv1D(32, 3, padding='same',activation='relu')(text_layer)
  text_layer = Conv1D(32, 3,padding='same', activation='relu')(text_layer) 
  text_layer = Conv1D(GENSIMtrainY.shape[-1], 3, padding='same',activation='sigmoid')(text_layer)
  model = Model(text_input_layer, text_layer)
  model.compile(optimizer=Adadelta(),
                    loss='categorical_crossentropy',
                    metrics=[dice_coef])
  return model

##**UNET**
https://youtu.be/wdi8HYhgGj0?t=1769

In [ ]:
# Функция по созданию сети UNET
def create_unet(k = 1, num_classes = 6, input_shape= (30, 300)):
    img_input = Input(input_shape) 

    # Блок 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k, 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    # Блок 2
    x = MaxPooling1D()(block_1_out)
    x = Conv1D(128 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    # Блок 3
    x = MaxPooling1D()(block_2_out)
    x = Conv1D(256 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # Блок 4
    x = MaxPooling1D()(block_3_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(256 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0020), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

# Создание xTrain и  yTrain используя Word2Vec GENSIM


In [ ]:
def pad_zeros(phrase, xLen = 256): # Функция для дополнением нулями
  while len(phrase) < xLen: # Пока сущестующий размер меньше чем требуемый 
    phrase.append([0] * embeddingSize) # Продолжаем добавлять элементы(нули) в фразу(zero-pad)  
  if len(phrase) > xLen: # В противном случае, если фраза больше чем  требуемый размер
    phrase = phrase[:xLen] # Берём только требуемое число элементов
  return phrase # Возвращаем обработанную фразу

# Создаём выборку
def getSets(model, senI, tagI):
  xVector = [] # Лист будет содержать ембеддинг представление на каждый индекс
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому лист-тексту
    tmp=[]
    for word in text: # Проходимся по кажому слову лист-текста
      try: # Если слово есть в модели (словарь не слишком мал), мы не получим ошибку
        tmp.append(model[word])
      except: # В противном случае пропустим слово
        pass

    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

In [ ]:
# Подаём лист листов слов в word2vec для обучающей выборке
# size = embeddingSize - Размер ембеддинга
# window = 10 -  минимальное расстояния между эмбеддинг словами
# min_count = 1 - Игнорируем все слова с частотой меньше чем 1
# workers = 10 -  Число потоков на обучение эмбеддинга
# iter = 20 -  Число эпох на обучение эмбеддинга
# max_vocab_size = 1e5 -  Число слов в "словарном запасе" word2vec
curTime = time.time()

modelGENSIM = word2vec.Word2Vec(xTrain + xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)
print('Обучение GENSIM модели заняло: ', round(time.time() - curTime, 2), 'с.')

Обучение GENSIM модели заняло:  71.82 с.


In [ ]:
modelGENSIM['договор'] # Пример превращения слова в вектор при помощи GENSIM

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 7.85699964e-01, -1.67415178e+00,  5.00308335e-01,  3.27189803e+00,
        1.10179663e+00,  6.91020908e-03,  1.36067164e+00, -1.72307837e+00,
        2.15302277e-02,  1.23110223e+00, -2.39618734e-01, -1.71559072e+00,
       -6.81143284e-01, -1.00146604e+00,  1.49691176e+00,  6.24539435e-01,
       -1.00925350e+00, -2.23928019e-01,  2.16359901e+00,  1.68185472e-01,
       -1.72086394e+00,  6.18731081e-01, -1.51999247e+00, -5.66072702e-01,
        2.46222541e-01, -6.65380538e-01, -1.72027183e+00, -1.18349445e+00,
       -5.75721145e-01, -2.03331709e-01,  2.21453398e-03,  1.68982792e+00,
       -6.65120482e-02,  5.94487667e-01, -9.89061236e-01,  4.35039818e-01,
        1.12598622e+00, -1.71992421e+00,  1.22951138e+00,  4.83514905e-01,
        6.65886939e-01, -3.68389517e-01,  2.42229915e+00, -3.45842212e-01,
       -1.97635198e+00, -2.19541097e+00,  1.80184603e-01, -1.13150311e+00,
        6.07044816e-01,  1.29831433e+00, -1.36832857e+00,  9.90896046e-01,
       -1.34510726e-01, -

In [ ]:
GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, xTrain, yTrain)
GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [ ]:
print('Размерности xTrain:', GENSIMtrainX.shape)
print('Размерности yTrain:', GENSIMtrainY.shape)

Размерности xTrain: (13299, 256, 300)
Размерности yTrain: (13299, 256, 6)


## **UNET**


### Стандартная архитектура

In [ ]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
model_b_UNET.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 256, 64)      57664       input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 256, 64)      256         conv1d_17[0][0]                  
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 256, 64)      0           batch_normalization_4[0][0]      
_______________________________________________________________________________________

In [ ]:
history = model_b_UNET.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=15, batch_size=64) # Feed the images divided among 2 classes to the model

Epoch 1/15
208/208 [==============================] - 27s 130ms/step - loss: 0.5511 - dice_coef: 0.5290 - val_loss: 0.7025 - val_dice_coef: 0.5034
Epoch 2/15
208/208 [==============================] - 26s 126ms/step - loss: 0.3423 - dice_coef: 0.6389 - val_loss: 0.6101 - val_dice_coef: 0.5680
Epoch 3/15
208/208 [==============================] - 26s 125ms/step - loss: 0.2842 - dice_coef: 0.6780 - val_loss: 0.5796 - val_dice_coef: 0.5685
Epoch 4/15
208/208 [==============================] - 26s 126ms/step - loss: 0.2589 - dice_coef: 0.6972 - val_loss: 0.8299 - val_dice_coef: 0.5573
Epoch 5/15
208/208 [==============================] - 26s 125ms/step - loss: 0.2427 - dice_coef: 0.7140 - val_loss: 0.7347 - val_dice_coef: 0.5869
Epoch 6/15
208/208 [==============================] - 26s 126ms/step - loss: 0.2395 - dice_coef: 0.7192 - val_loss: 0.6727 - val_dice_coef: 0.5842
Epoch 7/15
208/208 [==============================] - 26s 125ms/step - loss: 0.2268 - dice_coef: 0.7326 - val_loss: 1.

# **Проверяем работу сети**

In [ ]:
def recognizeSet(XX, YY, model, tags, length, value):
  correct_list = np.array([0] * 6) #Инициализируем массив правильных ответов в нули (сколько раз правильно определили класс)
  incorrect_list =  np.array([0] * 6)  #Инициализируем массив неправильных ответов в нули  (сколько раз неправильно определили класс)
  XX_array = XX
  YY_array = YY
  pred = model.predict(XX_array)
  pred[pred < value] = 0
  pred[pred > value] = 1

  for element in range(YY_array.shape[0]): # Проходим по всем примерам в батче
    for word in range(YY_array.shape[1]):  # Проходим по каждому слову
      for category in range(YY_array.shape[2]): # Проходим по каждой категории в слове
        if pred[element][word][category] == YY_array[element][word][category]: # Если предсказанное значение совпадает с истинным:
          correct_list[category] += 1 # Отмечаем, что мы правильно предсказали класс объекта
        else:  # Если предсказанное значение НЕ совпадает с истенным:
          incorrect_list[category] += 1 # Отмечаем, что мы не правильно предсказали класс объекта
      
  # Итоговая точность для каждого класса = кол. 100% * правильных/(кол. неправильных + кол. правильных)
  for i in range(6):
   print("Сеть распознала категорию  '{}' с точностью в {}%".format(tags[i], round(100*correct_list[i]/(correct_list[i] + incorrect_list[i]), 2)))
  total = round(100*np.mean(correct_list/(correct_list + incorrect_list)),2) 
  print("Средняя точность {}%".format(total))

In [ ]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
# s1 Условия
# s2 Запреты
# s3 Стоимость (услуг и прочее)
# s4 Условия (относительно дат - дедлайны)
# s5 Штрафы
# s6 Адреса и местоположения

In [ ]:
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_UNET, tags, xLen, 0.1) # Проверяем UNET архитектуру

Сеть распознала категорию  'S1' с точностью в 59.22%
Сеть распознала категорию  'S2' с точностью в 98.2%
Сеть распознала категорию  'S3' с точностью в 78.68%
Сеть распознала категорию  'S4' с точностью в 90.13%
Сеть распознала категорию  'S5' с точностью в 90.49%
Сеть распознала категорию  'S6' с точностью в 98.58%
Средняя точность 85.88%
